In [29]:
import pandas as pd
import wacgen
reload(wacgen)

<module 'wacgen' from 'wacgen.py'>

# Prepare test set

load and merge csv files

In [2]:
testpath = './SAIA_Data/test_set_ground_truth'
atest = pd.read_csv(testpath+'/ground_truth_a.csv')
btest = pd.read_csv(testpath+'/ground_truth_b.csv')
ctest = pd.read_csv(testpath+'/ground_truth_c.csv')

In [3]:
atest['Testset'] = 'A'
btest['Testset'] = 'B'
ctest['Testset'] = 'C'

In [4]:
alltest = atest.append(btest)
alltest = alltest.append(ctest)
len(alltest)

4500

In [5]:
alltest['prev_file'] = alltest['Image Number'].shift()
alltest['same'] = alltest['Image Number'] == alltest['prev_file']
alltest = alltest[alltest['same'] == False]
len(alltest)

1500

In [6]:
alltest.head()

,Abs. Location,Category,Color,Entry-Level,Identifier,Image Number,Other,Referring Expression,Rel. Location,Rel. Object,Size,Testset,prev_file,same
0,NaN,hat,NaN,hat,5383913d793954ad2d000035,1001_4.jpg,NaN,Hat,NaN,NaN,NaN,A,NaN,False
3,NaN,parrot,NaN,bird,537fb928d31907d16d000cac,10028_1.jpg,NaN,bbird,NaN,NaN,NaN,A,1001_4.jpg,False
6,NaN,hill,NaN,mountain,537fb7b5d31907d16d000b04,10065_4.jpg,NaN,mountain skyline outline,NaN,NaN,NaN,A,10028_1.jpg,False
9,NaN,dish,NaN,cake,537fc095d31907d16d001577,10143_1.jpg,NaN,cake,NaN,NaN,NaN,A,10065_4.jpg,False
12,NaN,painting,NaN,painting,53839350793954ad2d0000ea,10172_3.jpg,NaN,big painting on left of wall,prep_on_left,wall,big,A,10143_1.jpg,False


In [7]:
files = [int(i.split('_')[0]) for i in alltest['Image Number']]
regions = [int(i.split('_')[1][:-4]) for i in alltest['Image Number']]

alltest['file'] = files
alltest['region'] = regions

# Experiment 1: Non-interactive REG

In this experiment, we generate referring expressions with different wac (words-as-classifiers) models.
This model associates words with visual classifiers, and can be trained on different sets of visual features.
We will compare a basic set of visual features provided with the SAIA image corpus (features.mat) and features extracted with a ConvNet (googLE).

As this is a non-interactive experiment, we generate a single expression for each object in our test set, i.e. the optimal expression predicted by the generator.

## Apply generators

* we now generate refexp. for the RefeitGame
* if you want, you can skip this step and use the file
*EvalExp1/exp1_ground_truth_generated.csv*
* the following steps will show how to produce this file for the given generation models

load model trained on visual features provided with the saia corpus
(very basic colour and spatial features)

In [30]:
gensaia = wacgen.WacGen(saia=True,google=False,groundsplit=True)

Loaded classifiers ['* Started 2015-12-03_19:25\n', "Namespace(classf='logreg', feat_path='InData/features.mat', filter_rel=True, googLe=False, googSaia=False, just_splits=False, n_folds=10, n_neg=7, neg_from='random', out_path='OutData/Models/groundsplit_400saiafr7/', rex_path='InData/RealGames.txt', split_path='OutData/Splits/ground-truth-split.pklz', wrdl_crit=['most_common', '400'])"]


in addition to wac, we need a way to decide how many words the expression for a given object should contain
we do this in a very simple way and train some classifiers

In [31]:
_,_,lensaia = gensaia.get_len_classifiers()

Train len 1
Train len 2
Train len 3
Train len 4
Train len 5


In [33]:
saia_gen_list = []
for (ix,row) in alltest.iterrows():
    #print (row['file'],row['region'])
    gen_utt2 = gensaia.generate_beam_plen(lensaia,row['file'],row['region'])
    #print gen_utt2
    saia_gen_list.append(' '.join(gen_utt2[1:]))
    
alltest['saia'] = saia_gen_list

Now load the model based on visual features extracted with a ConvNet

In [34]:
gengoogle = wacgen.WacGen(saia=False,google=True,groundsplit=True)

Loaded classifiers ['* Started 2015-12-03_19:24\n', "Namespace(classf='logreg', feat_path='InData/features.mat', filter_rel=True, googLe=True, googSaia=False, just_splits=False, n_folds=10, n_neg=7, neg_from='random', out_path='OutData/Models/groundsplit_400gfr7/', rex_path='InData/RealGames.txt', split_path='OutData/Splits/ground-truth-split.pklz', wrdl_crit=['most_common', '400'])"]


In [35]:
_,_,lengoogle = gengoogle.get_len_classifiers()

Train len 1
Train len 2
Train len 3
Train len 4
Train len 5


In [36]:


google_gen_list = []
for (_,row) in alltest.iterrows():
    #print (row['file'],row['region'])
    try:
        gen_utt2 = gengoogle.generate_beam_plen(lengoogle,row['file'],row['region'])
    except:
        gen_utt2 = ['','']
    google_gen_list.append(' '.join(gen_utt2[1:]))
    
alltest['google'] = google_gen_list



In [37]:
google_glen_list = []
for (_,row) in alltest.iterrows():
    #print (row['file'],row['region'])
    try:
        gen_utt2 = gengoogle.generate_beam(row['file'],row['region'],len(row['Referring Expression'].split()))
    except:
        print "Could not generate", (row['file'],row['region'])
        gen_utt2 = ['','']
        
    google_glen_list.append(' '.join(gen_utt2[1:]))
    
alltest['google_glen'] = google_glen_list

/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


Could not generate (8103, 3)


/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


Could not generate (19579, 1)


In [38]:
alltest.rename(columns={'Referring Expression':'gold'},inplace=True)
allout = alltest[['Testset','file','region','gold','google','saia','google_glen']]
allout.to_csv('EvalExp1/exp1_ground_truth_generated.csv')

* *EvalExp1/exp1_ground_truth_generated.csv* is the output file that will be used by the evaluation scripts

## Run human evaluation

Now, you have to find some human subjects to evaluate the generated expressions.


You have to sit them in front of the computer and let them play the ReferitGame using the GUI
in EvalExp1.

**cd EvalExp1**

Just start the GUI as

**python evalgui_exp1.py -n N**
where N is the number of games the subject wants to play

this will produce log files stored in **EvalExp1/final_logs**


# Experiment 2: Installment-based REG

In this experiment, we generate referring expressions with a single wac (words-as-classifiers) model (based on googLE features), but using different modes of interactive delivery or presentation.

Thus, in case the user clicks on the wrong object in the first trial, we want to be able to correct or elaborate the refexp.

This is a more interactive experiment, but we will still generate all expressions for an object offline, according to two different strategies for installment-based REG.

Thus, the output file will have a slightly different format (additional columns for installments). And the GUI that present the refexp is changed accordingly.

## Apply generator

* as in Experiment 1, refexp. are generated off-line
* if you want, you can skip this step and use the file
*EvalExp2/exp2_ground_truth_generated.csv*
* the following steps will show how to produce this file for the given generation models

In [46]:
import wacgen_ia
reload(wacgen_ia)

<module 'wacgen_ia' from 'wacgen_ia.py'>

In [48]:
genia = wacgen_ia.WacGenIA()
genia.make_ia_classifiers()

Loaded word classifiers


wacgen_ia.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.testdf['file'] = files
wacgen_ia.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.testdf['region'] = regions


Loaded testfiles 1500
Excluded classifiers all ,tv ,to ,sorry ,gravel ,sitting ,blanket ,wooden ,peeps ,biker ,hanging ,second ,click ,what ,section ,man's ,full ,leg ,path ,greenery ,standing ,from ,two ,next ,this ,nearest ,grassy ,court ,1 ,tile ,beige ,okay ,structure ,spot ,plane ,end ,arch ,branches ,third ,coat ,over ,looks ,backpack ,main ,them ,dat ,half ,rocky ,beneath ,glasses ,spider ,sandy ,out ,rt ,3rd ,space ,looking ,foreground ,dirt ,where ,woman's ,first ,one ,feet ,directly ,open ,anyone ,their ,2 ,legs ,that ,shelf ,part ,distance ,and ,infront ,any ,shorts ,- ,so ,towards ,object ,most ,clear ,walking ,bright ,wearing ,his ,striped ,patch ,stuff ,ruins ,farthest ,seat ,see ,are ,close ,arm ,away ,3 ,hammock ,last ,womans ,whole ,2nd ,blonde ,three ,beer ,tallest ,empty ,near ,is ,it ,nightstand ,thingy ,things ,bowl ,beside ,wheel ,closest ,hand ,off ,i ,edge ,just ,photo ,hands ,mid ,yep ,bit ,has ,around ,couple ,whatever ,shoulder ,like ,piece ,facing ,either ,o

generate the expressions
(this will take a long time)

In [49]:
genia.generate_ia_distractors()
genia.generate_type_distractors()
att_l = genia.generate_ia()

Could not generate 8103 1
Could not generate 8103 2
Could not generate 8103 3
Could not generate 19579 1
Could not generate 19579 2
No Features for 8103 1
No Features for 8103 2
No Features for 8103 3
No Features for 19579 1
No Features for 19579 2
No distinguishing att expr found! 4872 8
No distinguishing att expr found! 16523 7
No distinguishing att expr found! 1735 12
No distinguishing att expr found! 5013 11
No distinguishing att expr found! 7401 3
No distinguishing att expr found! 8006 11


In [50]:
genia.testdf['ref_a1'] = [" ".join(u[0][1:]) for u in att_l]
genia.testdf['ref_a2'] = [" ".join(u[1][1:]) for u in att_l]
genia.testdf['ref_a3'] = [" ".join(u[2][1:]) for u in att_l]
genia.testdf['ref_h_a2'] = [" ".join(u[3][1:]) for u in att_l]
genia.testdf['ref_h_a3'] = [" ".join(u[4][1:]) for u in att_l]
genia.testdf['ref_h_nouns'] = [u[5] for u in att_l]
genia.testdf['ref_gentype'] = [u[6] for u in att_l]

gfile = './EvalExp2/exp2_ground_truth_generated.csv'
genia.testdf.to_csv(gfile)

## Run human evaluation

Now, you have to find some human subjects to evaluate the generated expressions.


You have to sit them in front of the computer and let them play the ReferitGame using the GUI
in EvalExp2.

**cd EvalExp2**

Just start the GUI as

* **python evalgui_ia.py -n N** (where N is the number of games the subject wants to play)

* **python evalgui_ia_bl.py -n N** (where N is the number of games the subject wants to play)

this will produce log files stored in **EvalExp2/final_logs**

For this evaluation, you need 2 groups of subjects:
 * g1: playing evalgui_ia
 * g2: playing evalgui_ia_bl
 

This is different in Experiment 1, where each subject sees refexp generated by different systems
(but the same overall non-interactive strategy).
 
Thus, each subject only sees 1 version of the interactive REG,
the baseline (evalgui_ia_bl) or the context-aware installments version (evalgui_ia).
